In [2]:
# init
from user_out import *

In [3]:
from collections.abc import Iterable
from fpylll import *
from Crypto.Util.number import long_to_bytes, bytes_to_long



sk = 3
n =  543
m = len(out)

def GaloisField(poly,vname = "y"):
    assert poly.base_ring() ==  GF(2) and poly.is_irreducible()
    return GF(2**poly.degree(),modulus = poly, name = vname)

def QuotientRing2GaloisField(qr):
    return GaloisField(qr.modulus())

def poly2int(poly):
    if poly not in PR: 
        poly = poly.lift()
    bits = poly.coefficients(sparse = False)
    return ZZ(bits,2)

def int2poly(num):
    num = ZZ(num)
    return PR(num.bits())

def check_all(M, target = [1,-1,0]):
    m,n = M.dimensions()
    res = []
    for i in range(m):
        # print(M[i])
        if all(ZZ(j) in target for j in M[i]) and (not all(ZZ(j) == 0 for j in M[i])):
            print(f"[+] {i+1}th vector is good")
            res.append(list(M[i]))
    return res

def build_modular_ssp_lattice(subset,target,mod):
    m = len(subset)
    n = RR(log(max(subset),2))
    M = matrix(ZZ, m+2, m+2)
    print(f"[+] density {RR(m/(n-1))}")
    for i in range(m):
        M[i,0] = subset[i]
        M[i,i+1] = 2
    M[m] = [mod] + [0]*(m+1)
    M[m+1]= [target] + [1]*(m+1)
    return M

def build_multiple_modular_ssp_lattice(subsets,targets,mods):
    m = len(subsets[0])
    k = len(subsets)
    ns = [RR(log(max(subset),2)) for subset in subsets]
    n = sum(ns)
    M = matrix(ZZ, m + 1 + k, m + 1 + k)
    print(f"[+] density {RR(m/n)}")
    for i in range(m):
        for j in range(k):
            M[i,j] = subsets[j][i]
        M[i,i+k] = 2
        
    for i in range(k):
        M[m + i,i] = mods[i]
    M[m+ k]= targets + [1]*(m + 1)
    return M

def solve_ssp(subset,target,mod,multiplier = 2**10,bkz = None):
    """
    if BKZ is None: we use LLL reduction
    else : BKZ is a list like [24,32,40]
    """
    subset = [ num*multiplier for num in subset]
    target *= multiplier
    mod *= multiplier
    M = build_modular_ssp_lattice(subset,target,mod)
    print(f"[+] lattice dimension {M.dimensions()}")
    if bkz == None:
        M = M.LLL()
        res = check_all(M)
        return res
    else:
        assert isinstance(bkz,Iterable), "bkz parameter should be a list of integers"
        bkz = list(bkz)
        bkz.sort()
        for bs in bkz:
            print(f"\n[+] try bkz with block_size = {bs}")
            %time M = M.BKZ(block_size = bs,  proof = False)
            res = check_all(M)
            if len(res) != 0:
                return res
            
def solve_multiple_ssp(subsets,targets,mods,multiplier = 2**10,bkz = None):
    """
    if BKZ is None: we use LLL reduction
    else : BKZ is a list like [24,32,40]
    """
    subsets = [ [num*multiplier for num in subset] for subset in subsets]
    targets = [multiplier*target for target in targets]
    mods = [multiplier*mod for mod in mods]
    M = build_multiple_modular_ssp_lattice(subsets,targets,mods)
    print(f"[+] lattice dimension {M.dimensions()}")
    if bkz == None:
        M = M.LLL()
        res = check_all(M)
        return res
    else:
        assert isinstance(bkz,Iterable), "bkz parameter should be a list of integers"
        bkz = list(bkz)
        bkz.sort()
        for bs in bkz:
            print(f"\n[+] try bkz with block_size = {bs}")
            # M = M.BKZ(block_size = bs, proof = False)
            M = M.BKZ(block_size = bs, proof = False, strategies = load_strategies_json(BKZ.DEFAULT_STRATEGY_PATH + BKZ.DEFAULT_STRATEGY))
            # M = M.BKZ(block_size = bs,algorithm = "NTL")
            res = check_all(M)
            if len(res) != 0:
                return res
            
def gen_ssp(n = 120, m = 90):
    secret = getrandbits(m)
    mod = random_prime(2**(n-1),2**n)
    subset = [randint(1,mod) for _ in range(m)]
    target = 0
    for i in range(m):
        if (secret>>i)&1:
            target += subset[i]
    return subset, target % mod, mod, secret

def test_ssp_solver(n = 180, m= 90):
    subset, target, mod, secret = gen_ssp(n,m)
    s = 0
    for i in range(m):
        if (secret>>i)&1:
            s+=subset[i]
    s %= mod
    print(f"[+] check ? {target == s}")
    print("[+] try LLL reduction")
    res = solve_ssp(subset, target, mod, 10)
    if len(res)==0:
        res = solve_ssp(subset, target, mod, 10, range(20,44,2))
    if len(res) == 0:
        print("no solution")
        return
    for row in res:
        mbits = [ 0 if r*row[-1] == 1 else 1 for r in row[1:-1]]
        _secret = ZZ(mbits,2)
        print(f"[+] solution { _secret = }, {_secret == secret}")
        
def Bytes_to_Bitvector(data, padlen = n):
    num = int.from_bytes(data, "big")
    return Int_to_Bitvector(num)

def Bitvector_to_Int(vec):
    return int("".join(map(str,list(vec))),2)

def Bitvector_to_Bytes(vec, padlen = None):
    if padlen == None:
        padlen = (len(vec) + 7)//8
    return Bitvector_to_Int(vec).to_bytes(padlen, "big")

def Int_to_Bitvector(num, padlen = n):
    ls = map(int, bin(num)[2:].zfill(padlen))
    return vector(GF(2),ls)
        
# test_ssp_solver()

In [14]:
# ISD decode
# eq_n = 800
M = matrix(GF(2),[Int_to_Bitvector(row, n) for row in ms])
print(M.dimensions())
c = Bytes_to_Bitvector(encoded_target)
C = codes.LinearCode(M.T)
D = codes.decoders.LinearCodeInformationSetDecoder(C, 20)
print(D.algorithm().time_estimate())
%time sol = D.decode_to_message(c)
err = M * sol  - c
print(f"{list(err).count(1) = }")
target_val = Bitvector_to_Int(sol)

(800, 543)
2017723.5873039477
CPU times: user 1min 2s, sys: 1e+03 ns, total: 1min 2s
Wall time: 1min 2s
list(err).count(Integer(1)) = 10


In [4]:
PR.<x> = PolynomialRing(GF(2))
fs = factor(x**n - 1)
print(fs)
mods = []
subsets = []
targets = []

for i in range(1,sk+1):
    mod = fs[-i][0]
    print(f"[+] {i}th {mod.degree() = } {m = }")
    GFT = GaloisField(mod)
    O = GFT.order() - 1
    g =  GFT.primitive_element()
    polys = [int2poly(s) for s in out + [target_val]]
    poly_list = [GFT(poly % GFT.modulus()) for poly in polys]
    dlogs = [discrete_log(poly,g,ord = O) for poly in poly_list]
    subset =dlogs[:m]
    target = dlogs[-1]
    mod = O
    mods.append(mod)
    subsets.append(subset)
    targets.append(target)

(x + 1) * (x^2 + x + 1) * (x^180 + x^177 + x^175 + x^173 + x^170 + x^167 + x^162 + x^161 + x^160 + x^159 + x^158 + x^157 + x^155 + x^154 + x^153 + x^152 + x^150 + x^148 + x^147 + x^146 + x^145 + x^139 + x^133 + x^131 + x^130 + x^128 + x^124 + x^123 + x^122 + x^120 + x^119 + x^115 + x^114 + x^113 + x^112 + x^111 + x^110 + x^106 + x^105 + x^103 + x^99 + x^98 + x^95 + x^94 + x^93 + x^91 + x^89 + x^87 + x^81 + x^77 + x^74 + x^71 + x^70 + x^69 + x^67 + x^66 + x^65 + x^64 + x^63 + x^61 + x^60 + x^59 + x^58 + x^57 + x^55 + x^53 + x^52 + x^49 + x^45 + x^43 + x^41 + x^37 + x^34 + x^32 + x^29 + x^26 + x^23 + x^20 + x^18 + x^17 + x^14 + x^13 + x^12 + x^11 + x^10 + x^9 + x^5 + x + 1) * (x^180 + x^179 + x^175 + x^171 + x^170 + x^169 + x^168 + x^167 + x^166 + x^163 + x^162 + x^160 + x^157 + x^154 + x^151 + x^148 + x^146 + x^143 + x^139 + x^137 + x^135 + x^131 + x^128 + x^127 + x^125 + x^123 + x^122 + x^121 + x^120 + x^119 + x^117 + x^116 + x^115 + x^114 + x^113 + x^111 + x^110 + x^109 + x^106 + x^10

In [5]:
res = solve_multiple_ssp(subsets, targets, mods, 14, range(20,100,4))

[+] density 0.333712645759734
[+] lattice dimension (188, 188)

[+] try bkz with block_size = 20

[+] try bkz with block_size = 24

[+] try bkz with block_size = 28

[+] try bkz with block_size = 32

[+] try bkz with block_size = 36

[+] try bkz with block_size = 40

[+] try bkz with block_size = 44

[+] try bkz with block_size = 48

[+] try bkz with block_size = 52

[+] try bkz with block_size = 56

[+] try bkz with block_size = 60

[+] try bkz with block_size = 64
[+] 1th vector is good


In [7]:
print(res)
for row in res:
    mbits = [ 0 if r*row[-1] == 1 else 1 for r in row[sk:-1]]
    secret = ZZ(mbits,2)  
    print(f"[+] solution { secret = }")

[[0, 0, 0, -1, 1, -1, 1, -1, -1, -1, 1, 1, -1, 1, 1, 1, 1, -1, 1, 1, -1, 1, 1, 1, 1, 1, -1, -1, 1, 1, 1, -1, -1, -1, -1, -1, 1, 1, -1, 1, 1, -1, -1, 1, 1, -1, -1, -1, 1, 1, 1, 1, 1, -1, -1, 1, -1, 1, -1, 1, 1, -1, 1, -1, 1, -1, -1, 1, -1, -1, -1, 1, 1, -1, -1, 1, -1, -1, 1, 1, -1, -1, 1, 1, 1, 1, 1, 1, -1, 1, -1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, 1, -1, 1, -1, -1, -1, 1, -1, -1, -1, -1, -1, -1, -1, 1, -1, -1, 1, 1, -1, -1, -1, 1, -1, -1, -1, -1, 1, -1, -1, 1, -1, -1, -1, 1, -1, 1, 1, -1, 1, 1, -1, 1, 1, 1, -1, -1, 1, -1, -1, -1, -1, 1, -1, 1, 1, -1, -1, -1, -1, 1, 1, 1, -1, -1, 1, -1, 1, -1, 1, -1, 1, -1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1]]
[+] solution  secret = 9809570453264748617729173272798292415295067216928719477


In [15]:
D

Information-set decoder (Lee-Brickell) for [800, 543] linear code over GF(2) decoding up to 20 errors 